In [3]:
import openai
import pandas as pd
import os

In [4]:
openai.api_key = os.environ['GPT_OPEN_AI_KEY']

In [5]:
def prepare_dataset():
    # Example dataset
    data = [
        {"prompt": "User: Hi, how can I reset my password?\nAssistant:", "completion": " You can reset your password by going to the settings page and clicking on 'Forgot Password'."},
        {"prompt": "User: What are your business hours?\nAssistant:", "completion": " Our business hours are Monday to Friday, 9 AM to 5 PM."},
        # Add more examples as needed
    ]
    return data

In [6]:
# Step 2: Fine-tune the model
def fine_tune_model(training_data):
    response = openai.FineTune.create(
        training_file=training_data,
        model="davinci",  # Choose the model variant
        n_epochs=4,
        batch_size=4,
        learning_rate_multiplier=0.1,
        prompt_loss_weight=0.1,
        compute_classification_metrics=False
    )
    return response

In [7]:
# Step 3: Evaluate the model
def evaluate_model(model_id, test_data):
    results = []
    for test in test_data:
        response = openai.Completion.create(
            model=model_id,
            prompt=test['prompt'],
            max_tokens=50,
            temperature=0.7,
            stop=["\n"]
        )
        result = {
            'prompt': test['prompt'],
            'expected': test['completion'],
            'generated': response.choices[0].text.strip()
        }
        results.append(result)
    return results

In [8]:
# Step 4: Deploy and monitor the model
def deploy_model(model_id):
    # Deployment will depend on the specific application platform
    # Here we simulate the deployment by using the model in a chat function
    def chat(prompt):
        response = openai.Completion.create(
            model=model_id,
            prompt=prompt,
            max_tokens=50,
            temperature=0.7,
            stop=["\n"]
        )
        return response.choices[0].text.strip()

    return chat

In [9]:
# Step 1: Prepare the dataset
training_data = prepare_dataset()

# Convert training data to pandas DataFrame and save as CSV (for upload to OpenAI)
df = pd.DataFrame(training_data)
df.to_csv('training_data.csv', index=False)

# Step 2: Fine-tune the model
fine_tune_response = fine_tune_model('training_data.csv')
model_id = fine_tune_response['id']

# Wait for the fine-tuning to complete
import time
while True:
    status = openai.FineTune.retrieve(model_id)
    if status['status'] == 'succeeded':
        break
    elif status['status'] == 'failed':
        raise Exception("Fine-tuning failed")
    time.sleep(60)  # Wait for 1 minute before checking again

# Step 3: Evaluate the model
test_data = prepare_dataset()  # In a real scenario, this should be a separate test dataset
evaluation_results = evaluate_model(model_id, test_data)
for result in evaluation_results:
    print(f"Prompt: {result['prompt']}")
    print(f"Expected: {result['expected']}")
    print(f"Generated: {result['generated']}\n")

# Step 4: Deploy the model
chat = deploy_model(model_id)
while True:
    user_input = input("User: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    response = chat(f"User: {user_input}\nAssistant:")
    print(f"Assistant: {response}")

APIRemovedInV1: 

You tried to access openai.FineTune, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
